# <span style='color:blue'> RBDO with OpenTURNS </span>

RBDO class is used to solve such reliability-based design optimization problem:

\begin{eqnarray}
\min_{\mathbf{d},\mathbf{p}} & & C(\mathbf{d},\mathbf{p}) \\
\text{s.t.} & & \left\{
                \begin{array}{ll}
                  \mathbb{P} \left(g_i(\mathbf{d},\mathbf{X}(\mathbf{p}),\mathbf{Z})\leq 0 \right) \leq P_{f_i}^T, \;\; i= 1,\ldots,m\\
                  h_j(\mathbf{d},\mathbf{p})\leq 0, \;\; j= 1,\ldots,M\\
                \end{array}
              \right.
\end{eqnarray}
with $\mathbf{d} \in \mathbb{R}^{n_d} $, $\mathbf{p}\in\mathbb{R}^{n_p}$, $\mathbf{X}\sim \mathbf{f}_{\mathbf{X}|\mathbf{d}}(\cdot)$ of dimension $n_X$ and $\mathbf{Z}\sim \mathbf{f}_{\mathbf{Z}}(\cdot)$

- $\mathbf{d}$ is a vector of deterministic variables
- $\mathbf{p}$ is a vector of deterministic variables corresponding to hyperparameters of the random variables $\mathbf{X}$ defined by its joint PDF $\mathbf{f}_{\mathbf{X}|\mathbf{d}}(\cdot)$ (e.g., mean, standard deviation, shape parameters) 
- $\mathbf{X}$ is a random vector with some hyperparameters defining its PDF that can depend on $\mathbf{p}$
- $\mathbf{Z}$ is a random vector with fixed hyperparameters defined by the PDF $\mathbf{f}_{\mathbf{Z}}(\cdot)$
- $P_{f_i}^T$ is a threshold of maximal probability value for the constraint $i$


Three algorithms are currently available in RBDO class to solve such a problem:
- <span style='color:blue'> RIA </span>  (Reliability Index Approach) algorithm
- <span style='color:blue'> PMA </span> (Performance Measure Approach) algorithm with 4 different methods for inverse FORM: 'SLSQP', 'AMV', 'CMV' and 'HMV'
- <span style='color:blue'> SORA </span> (Sequential Optimization and Reliability Assessment) algorithm with 4 different methods for inverse FORM: 'SLSQP', 'AMV', 'CMV' and 'HMV'

Please refer to the following papers for more details on the algorithms:
- Valdebenito, Marcos A., and Gerhart I. Schuëller. "A survey on approaches for reliability-based optimization." Structural and Multidisciplinary Optimization 42.5 (2010): 645-663.
- Aoues, Younes, and Alaa Chateauneuf. "Benchmark study of numerical methods for reliability-based design optimization." Structural and multidisciplinary optimization 41.2 (2010): 277-294.
- Du, Xiaoping, and Wei Chen. "Sequential optimization and reliability assessment method for efficient probabilistic design." J. Mech. Des. 126.2 (2004): 225-233.

To illustrate the RBDO class, let's consider the following RBDO problem:

\begin{eqnarray}
\min_{d_0,p_0,p_1} & & 2+(p_0-1)^2+(0.7-p_1-d_0)^2 \\
\text{s.t.} & & \left\{
                \begin{array}{ll}
                  \mathbb{P} \left(g_i(d_0,\mathbf{X}(p_0,p_1),Z_0)\leq 0 \right) \leq P_{f_i}^T, \;\; i= 1,\ldots,2\\
                  0.1 \leq d_0\leq 1.5 \\
                  0.1 \leq p_0\leq 1.5 \\
                  0.1 \leq p_1\leq 1.5 \\
                \end{array}
              \right.
\end{eqnarray}
with:
- $g_1(\mathbf{X}) = 1-d_0*(X_1+1)^2+(Z_0+2)^2-X_2$ 
- $g_2(\mathbf{X}) = (1/5)*(1+d_0)^2+X_2-Z_0+3.0$

and $\mathbf{X}=[X_1,X_2]$ a random vector of two independant Gaussian variables $X_1\sim \mathcal{N}(p_0,0.5),X_2\sim \mathcal{N}(p_1,0.5)$, $[p_0,p_1] = [\mu_{X_1},\mu_{X_2}]$ the mean values of the random variables $X_1,X_2$ and $Z_0\sim \mathcal{N}(2.,0.4)$ a random variable.
For the probability target maximal, it is assumed $P_{f_i}^T = 0.01$.

In [1]:
import openturns as ot
from RBDO_class import *
import time

## Definition of the objective Python function
def obj(d,p):
    d0 = d[0]
    p0 = p[0]
    p1 = p[1]
    y = 2+(p0-1.)**2+(0.7-p1-d0)**2        
    return y

## Definition of the objective RBDO function with active d and p on the objective function. 
## In the example, no d are given (no active d), and the objective function depends on p=[p_1,p_2] (both active on the function),
## therefore in the vector XX=[d,p] p_1,p_2 are first and second index of XX
active_index_d = [0]
active_index_p = [1,2]
len_XX = 3
f_obj = Objective(obj,active_index_d,active_index_p,len_XX)


## Definition of the constraint functions
def g1(d,x,z):
    d0 = d[0]
    
    x1 = x[0]
    x2 = x[1]
    
    z0 = z[0]
    
    g1 = 1.0-d0*(x1+1)**2+(z0+2)**2-x2
    return [g1]

def g2(d,x,z):
    d0 = d[0]
    
    x2 = x[0]
    
    z0 = z[0]
    g2 = (1./5.)*(1+d0)**2+x2-z0+3.0
    return [g2]

## Definition of the parametric distributions for X for each constraint
def updatedistg1(p):
    
    m_1 = p[0]
    m_2 = p[1]
    marg1 = ot.Normal(m_1,0.5)
    marg2 = ot.Normal(m_2,0.5)
    dist = ot.ComposedDistribution([marg1,marg2])
    return dist

z0 = ot.Normal(2.,0.4)
distZ_g1 = z0

def updatedistg2(p):
    
    m_2 = p[0]
    marg2 = ot.Normal(m_2,0.5)
    dist = marg2
    return dist

distZ_g2 = z0

### RIA algorithm

In [2]:
active_index_d_g1 = [0] 
active_index_p_g1 = [1,2] 
PfT_g1 = 0.01 
PIneq_g1 = PIneqCons(g1,active_index_d_g1,active_index_p_g1,updatedistg1,distZ_g1,PfT_g1)

active_index_d_g2 = [0] 
active_index_p_g2 = [2] 
PfT_g2 = 0.01 
PIneq_g2 = PIneqCons(g2,active_index_d_g2,active_index_p_g2,updatedistg2,distZ_g2,PfT_g2)

## Definition of the list of constraints
PIneqCons_list = [PIneq_g1,PIneq_g2]  #list of Probability inequality constraints
DIneqCons_list = [] #list of deterministic constraints --> no deterministic ,constraints here
Bounds = ot.Interval([0.1,0.1,0.1], [1.5,1.5,1.5])  #Definition of the bounds on XX=[d,p] 
len_d = 1 #len of deterministic variable vector d
len_p = 2 #len of hyperparemeters variable vector p

# Definition of RBDO problem using the previously defined objective function, constraint functions and bounds
RBDO_Problem_test = RBDO_Problem(f_obj,PIneqCons_list,DIneqCons_list,Bounds,len_d,len_p)

# Initial optimization points
InitialPoint = [0.7,0.7,0.7]

# Solver type of RIA
Solver = ot.NLopt('LN_COBYLA')
Solver.setVerbose(True)
Solver.setMaximumEvaluationNumber(1000)
Solver.setMaximumIterationNumber(200)

t0 = time.time()
# Instantiation of the RIA algorithm
RIA_Algorithm_test = RIA_Algorithm(RBDO_Problem_test,Solver,InitialPoint)

# Run of RIA algorithm
result = RIA_Algorithm_test.run()

#Obtained results
print('Optimal design variables p = ',RIA_Algorithm_test.get_optimum())
print('Optimal objective function value = ',RIA_Algorithm_test.get_foptimum())
print('Optimal constraint functions value = ',RIA_Algorithm_test.get_consoptimum())
print('Number of RIA iterations = ', result.getEvaluationNumber())
print('Time CPU = ',time.time()-t0)

Optimal design variables p =  [0.987402,0.253786,1.5]
Optimal objective function value =  [5.75164]
Optimal constraint functions value =  [0.009999990752357385, 0.010000004002743097]
Number of RIA iterations =  42
Time CPU =  0.8829998970031738


### PMA algorithm

In [3]:
active_index_d_g1 = [0] 
active_index_p_g1 = [1,2] 
PfT_g1 = 0.01 
InvFORM_solver_g1 = 'SLSQP' ## Choice of Inverse FORM Solver : ['SLSQP','AMV','CMV','HMV']
PIneq_g1 = PIneqCons(g1,active_index_d_g1,active_index_p_g1,updatedistg1,distZ_g1,PfT_g1,solver_invFORM=InvFORM_solver_g1)

active_index_d_g2 = [0] 
active_index_p_g2 = [2] 
PfT_g2 = 0.01 
InvFORM_solver_g2 = 'SLSQP' ## Choice of Inverse FORM Solver : ['SLSQP','AMV','CMV','HMV']
PIneq_g2 = PIneqCons(g2,active_index_d_g2,active_index_p_g2,updatedistg2,distZ_g2,PfT_g2,solver_invFORM=InvFORM_solver_g2)

## Definition of the list of constraints
PIneqCons_list = [PIneq_g1,PIneq_g2]  #list of Probability inequality constraints
DIneqCons_list = [] #list of deterministic constraints --> no deterministic ,constraints here
Bounds = ot.Interval([0.1,0.1,0.1], [1.5,1.5,1.5])  #Definition of the bounds on XX=[d,p] 
len_d = 1 #len of deterministic variable vector d
len_p = 2 #len of hyperparemeters variable vector p

# Definition of RBDO problem using the previously defined objective function, constraint functions and bounds
RBDO_Problem_test = RBDO_Problem(f_obj,PIneqCons_list,DIneqCons_list,Bounds,len_d,len_p)

# Initial optimization points
InitialPoint = [0.7,0.7,0.7]

# Solver type of PMA
Solver = ot.NLopt('LN_COBYLA')
Solver.setVerbose(True)
Solver.setMaximumEvaluationNumber(1000)
Solver.setMaximumIterationNumber(200)

t0 = time.time()
# Instantiation of the PMA algorithm
PMA_Algorithm_test = PMA_Algorithm(RBDO_Problem_test,Solver,InitialPoint)

# Run of PMA algorithm
result = PMA_Algorithm_test.run()

#Obtained results
print('Optimal design variables p = ',PMA_Algorithm_test.get_optimum())
print('Optimal objective function value = ',PMA_Algorithm_test.get_foptimum())
print('Optimal constraint functions value = ',PMA_Algorithm_test.get_consoptimum())
print('Number of PMA iterations = ', result.getEvaluationNumber())
print('Time CPU = ',time.time()-t0)

Optimal design variables p =  [0.987404,0.253776,1.5]
Optimal objective function value =  [5.75166]
Optimal constraint functions value =  [0.010000000060678289, 0.009999999995935524]
Number of PMA iterations =  39
Time CPU =  0.5160000324249268


### SORA algorithm

In [4]:
active_index_d_g1 = [0] 
active_index_p_g1 = [1,2] 
PfT_g1 = 0.01 
InvFORM_solver_g1 = 'SLSQP' ## Choice of Inverse FORM Solver : ['SLSQP','AMV','CMV','HMV']
PIneq_g1 = PIneqCons(g1,active_index_d_g1,active_index_p_g1,updatedistg1,distZ_g1,PfT_g1,solver_invFORM=InvFORM_solver_g1)

active_index_d_g2 = [0] 
active_index_p_g2 = [2] 
PfT_g2 = 0.01 
InvFORM_solver_g2 = 'SLSQP' ## Choice of Inverse FORM Solver : ['SLSQP','AMV','CMV','HMV']
PIneq_g2 = PIneqCons(g2,active_index_d_g2,active_index_p_g2,updatedistg2,distZ_g2,PfT_g2,solver_invFORM=InvFORM_solver_g2)

## Definition of the list of constraints
PIneqCons_list = [PIneq_g1,PIneq_g2]  #list of Probability inequality constraints
DIneqCons_list = [] #list of deterministic constraints --> no deterministic ,constraints here
Bounds = ot.Interval([0.1,0.1,0.1], [1.5,1.5,1.5])  #Definition of the bounds on XX=[d,p] 
len_d = 1 #len of deterministic variable vector d
len_p = 2 #len of hyperparemeters variable vector p

# Definition of RBDO problem using the previously defined objective function, constraint functions and bounds
RBDO_Problem_test = RBDO_Problem(f_obj,PIneqCons_list,DIneqCons_list,Bounds,len_d,len_p)

# Initial optimization points
InitialPoint = [0.7,0.7,0.7]

# Solver type of SORA
Solver = ot.NLopt('LN_COBYLA')
Solver.setVerbose(True)
Solver.setMaximumEvaluationNumber(1000)
Solver.setMaximumIterationNumber(200)

# Instantiation of the SORA algorithm
SORA_Algorithm_test = SORA_Algorithm(RBDO_Problem_test,Solver,InitialPoint)

t0 = time.time()
# Run of PMA algorithm
result = SORA_Algorithm_test.run()

#Obtained results
print('Optimal design variables p = ',SORA_Algorithm_test.get_optimum())
print('Optimal objective function value = ',SORA_Algorithm_test.get_foptimum())
print('Optimal constraint functions value = ',SORA_Algorithm_test.get_consoptimum())
print('Number of SORA iterations = ', result.getEvaluationNumber())
print('Time CPU = ',time.time()-t0)

Optimal design variables p =  [0.987404,0.253776,1.5]
Optimal objective function value =  [5.75166]
Optimal constraint functions value =  [0.010000000058496362, 0.010000000000419628]
Number of SORA iterations =  34
Time CPU =  0.09100008010864258
